# 第六章：SEC 知识图谱扩展

- 在本章中，我们将引入第二个SEC数据集，以扩展原始提交表格的上下文。
- 第二组表格提供了关于机构投资经理及其持有公司权益的信息。
- 通过将这些数据添加到图谱中，我们将能够对组合数据集提出更复杂的问题，以帮助理解市场动态。

# 一、环境配置

本教程使用 OpenAI 所开放的 ChatGPT API，因此您需要首先拥有一个 ChatGPT 的 API_KEY（也可以直接访问官方网址在线测试），然后需要安装 OpenAI 的第三方库。为了兼顾简便与兼容性，本教程将介绍在 ```Python 3``` 环境中基于 ```openai.api_key``` 方法的配置。另有基于环境变量的配置方法，详情请参考 [OpenAI 官方文档](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)。

首先需要安装 OpenAI，LangChain等工具库：
```bash
pip install openai langchain langchain_community langchain_openai
```

- 首先我们要导入一些Python包，并设置一些全局变量，以便在笔记本中使用。

## 1.1 导入第三方库

In [ ]:
import os
import textwrap

# OpenAI
import openai

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

# Warning control
import warnings
warnings.filterwarnings("ignore")

## 1.2 设置环境变量

In [ ]:
# 设置 API_KEY, 请替换成您自己的 API_KEY
openai.api_key = "sk-..."

# 设置 Neo4j, 请替换成您自己的 Neo4j_XXX
NEO4J_URI = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""
NEO4J_DATABASE = 'neo4j'

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [3]:
# 创建一个 Neo4jGraph 实例来连接到 Neo4j 数据库
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=NEO4J_DATABASE
)

# 二、数据加载

- SEC From 13 是由机构投资管理公司提交的，用来报告它们投资的上市公司。这些表格以 XML 文件的形式提供。
- 在数据准备过程中，我们从 XML 中提取了特定字段，并将其添加到 CSV 文件中的一行。

### 2.1 查看数据
- 投资管理公司必须通过提交名为 **Form 13** 的文件向 SEC 报告其对公司的投资
- 您将为已投资 NetApp 的经理加载 Form 13 的集合
- 您可以在数据目录来查看 CSV 文件（./data/form13.csv）

- 首先，我们可以使用 csv.dictreader 读取CSV文件，它会解析每一行并将其转换为使用 csv 头行作为键的字典。

In [4]:
import csv

all_form13s = []

with open('./data/form13.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader: # each row will be a dictionary
      all_form13s.append(row)

- 让我们快速看一下这些行的样子，也许只看前五行。我们可以看到这些公司都投资于同一家公司。
- 如果我们看看公司名称，有 NetApp，再次出现 NetApp 。这些管理公司有不同的名字，但它们都是 NetApp 的投资者。
- 我们可以看到有关于公司本身的详细信息，比如经理的名字、经理的地址，以及这个中央索引键。还有关于它们所做投资的具体信息，比如报告日历是什么，价值是多少，股票数量是多少。这些都是合理的
- 在这里，价值是货币价值，表示美元。我们还可以看到一些关于他们投资的公司的元数据，包括 cusip 代码和 cusip6 代码。

In [5]:
all_form13s[0:5]

[{'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
  'managerCik': '1000275',
  'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
  'managerName': 'Royal Bank of Canada',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '64395000000.0',
  'shares': '842850'},
 {'source': 'https://sec.gov/Archives/edgar/data/1002784/0001387131-23-009542.txt',
  'managerCik': '1002784',
  'managerAddress': '1875 Lawrence Street, Suite 300, Denver, CO, 80202-1805',
  'managerName': 'SHELTON CAPITAL MANAGEMENT',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '2989085000.0',
  'shares': '39124'},
 {'source': 'https://sec.gov/Archives/edgar/data/1007280/0001007280-23-000008.txt',
  'managerCik': '1007280',
  'managerAddress': '277 E TOWN ST, COLUMBUS, OH, 43215',
  'managerName': 'PUBLIC 

- 让我们看看有多少行。我们会检查长度。

In [6]:
len(all_form13s)

561

- 好的，有561行。我们预计会创建561家公司。

# 三、图谱构建

- 现在从每一行中，我们将创建两个节点，一个是管理公司，一个是它们投资的公司。
- 管理公司节点将有一个经理标签。它们将基于SEC的中央索引键唯一，并且还会有一个经理名称属性。
- 公司节点将有一个公司标签，它们将基于 cusip6 标识符唯一。
- 公司节点还会从 Form 13 数据中获得一个公司名称和一个完整的 cusip6 属性。

首先，让我们创建公司节点。

### 3.1 创建公司节点

- 我们合并具有公司标签且唯一由 cusip6 标识符标识的公司节点。
- 我们看到，在创建时，我们将设置公司名称和 cusip6 编号。

In [7]:
# 现在只使用第一个表格
first_form13 = all_form13s[0]

cypher = """
MERGE (com:Company {cusip6: $cusip6})
  ON CREATE
    SET com.companyName = $companyName,
        com.cusip = $cusip
"""

kg.query(cypher, params={
    'cusip6':first_form13['cusip6'], 
    'companyName':first_form13['companyName'], 
    'cusip':first_form13['cusip'] 
})

[]

快速检查一下合理性，我们预计创建的公司是NetApp。

In [8]:
cypher = """
MATCH (com:Company)
RETURN com LIMIT 1
"""

kg.query(cypher)

[{'com': {'cusip': '64110D104',
   'companyName': 'NETAPP INC',
   'cusip6': '64110D'}}]

- 我们已经在知识图谱中为 NetApp 创建了 Form10k 表格。我们可以通过找到基于 cusip6 标识符的节点对来匹配新创建的公司节点和相关的 Form10k 表格。我们在这里所做的就是匹配一个公司和一个表格，这两个节点的 cusip6 相同，然后返回这两个节点。
- 我们可以再次运行这个匹配，但现在因为我们注意到表格名称，表格中的公司名称不仅显示 NetApp Inc，还显示了名称的变体。

In [9]:
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  RETURN com.companyName, form.names
"""

kg.query(cypher)

[{'com.companyName': 'NETAPP INC', 'form.names': ['Netapp Inc', 'NETAPP INC']}]

- 我们可以将这些值提取到公司节点以丰富它。我们会根据 cusip6 进行匹配，然后一旦得到匹配，我们将设置公司的名称为表格的名称。

In [10]:
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  SET com.names = form.names
"""

kg.query(cypher)

[]

In [11]:
kg.query("""
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  MERGE (com)-[:FILED]->(form)
""")


[]

### 3.2 创建经理节点

- 进一步，通过从公司到表格的配对，我们现在将创建一个关系，以便我们知道这个公司提交了这个表格。
- 投资经理节点将有一个 manager 标签，接下来让我们创建这些节点。
- 我们有一个带有 manager 标签的经理节点，我们希望它根据经理的 CiK 编号是唯一的。
- 在创建时，我们将设置经理名称和经理地址。像之前一样，我们将传递一个字典到 manager 参数中，这将是用于在这个查询中创建这些节点的查询参数。
- 我们首先仅为第一个 Form 13 这样做。

In [12]:
cypher = """
  MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""

kg.query(cypher, params={'managerParam': first_form13})

[]

快速检查一下合理性，确保我们做得正确。

In [13]:
kg.query("""
  MATCH (mgr:Manager)
  RETURN mgr LIMIT 1
""")

[{'mgr': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'}}]

### 3.3 创建唯一性约束以避免重复的经理

- 管理公司会有很多，多达 561个。所以我们创建一个唯一性约束，以避免意外创建重复节点。

In [14]:
kg.query("""
CREATE CONSTRAINT unique_manager 
  IF NOT EXISTS
  FOR (n:Manager) 
  REQUIRE n.managerCik IS UNIQUE
""")

[]

### 3.4 创建经理姓名的全文索引以启用文本搜索

- 另外，我们还可以在经理节点上创建一个全文索引。全文索引对于关键字搜索非常有用。如果我们考虑向量索引，它允许我们基于相似概念进行搜索。全文索引允许基于相似字符串进行搜索。我们可以直接查询全文索引，就像我们可以直接查询向量索引一样。

In [15]:
kg.query("""
CREATE FULLTEXT INDEX fullTextManagerNames
  IF NOT EXISTS
  FOR (mgr:Manager) 
  ON EACH [mgr.managerName]
""")


[]

- 在这里，我们将搜索 "royal bank"。查询将返回一个节点和一个分数，就像向量搜索一样。如果匹配，我们会找到节点管理器名称以及分数。所以 "royal" 和 "bank" 都找到了不错的匹配项。但最佳匹配当然是加拿大皇家银行。

In [16]:
kg.query("""
  CALL db.index.fulltext.queryNodes("fullTextManagerNames", 
      "royal bank") YIELD node, score
  RETURN node.managerName, score
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 0.2615291476249695}]

- 现在我们准备为CSV文件中出现的所有管理公司创建节点。只需使用Python遍历所有行。
- 所以在参数中，我们将经理参数设置为任何Form 13。我们知道这是一个字典，因为所有Form 13都是字典列表。

In [17]:
cypher = """
  MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""
# loop through all Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'managerParam': form13 })

再次检查合理性，所以我们将匹配所有经理并返回这些经理的计数。我们预计有561个。

In [18]:
kg.query("""
    MATCH (mgr:Manager) 
    RETURN count(mgr)
""")

[{'count(mgr)': 561}]

### 3.5 在经理和公司之间建立关系

现在我们可以使用 Form 13 CSV 中的信息找到管理节点和公司节点的配对。
- 在这个查询中，我们可以看到我们将传递一个名为investment parameter的查询参数。
- 在匹配中，我们将匹配investment parameter的manager CIK，然后找到同一行中的公司，我们将找到investment parameter的 cusip6。所以这个匹配将基于这两个值找到一个匹配该行的经理和公司。如果我们记得我们的第一个真实数据，那就是加拿大皇家银行。当然，它们投资了NetApp。

In [19]:
cypher = """
  MATCH (mgr:Manager {managerCik: $investmentParam.managerCik}), 
        (com:Company {cusip6: $investmentParam.cusip6})
  RETURN mgr.managerName, com.companyName, $investmentParam as investment
"""

kg.query(cypher, params={ 
    'investmentParam': first_form13 
})

[{'mgr.managerName': 'Royal Bank of Canada',
  'com.companyName': 'NETAPP INC',
  'investment': {'shares': '842850',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'value': '64395000000.0',
   'cusip6': '64110D',
   'cusip': '64110D104',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'managerCik': '1000275'}}]

- 我们可以找到管理节点和它们投资的公司。很好。现在我们可以将这些节点连接起来。这是我们在课程中之前做过的事情，但查询会有点长。所以让我们逐行解析它。
- 我们要创建一些 Cypher。第一行我们想要的是我们之前的确切匹配，所以现在我们有一个经理和它们投资的相关公司。所以我们想要在它们之间创建一个关系，我们将使用 merge。
- 我们将从那个经理通过一个 ownStockin 关系合并到公司。我们希望ownStockin是唯一的，以防它们有多次投资。
- 如果我们还记得，查看CSV文件时，一些行有一个报告日历或季度值。让我们使用这个作为我们要创建的ownStockin关系的唯一属性。
- 属性将被称为报告日历或季度。我们将从查询参数中获取它。我们之前见过这个。在关系创建时，我们还将创建这些额外的值。
- 最后，我们将从经理、创建的关系和公司的名称中返回几个属性。
- 当我们用KG查询调用它时，要传递的参数叫做 ownsParam。这是我们在这里一直使用的。我们现在将只使用第一个Form 13。

In [20]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN { 
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter
}]->(com)
ON CREATE
    SET owns.value  = toFloat($ownsParam.value), 
        owns.shares = toInteger($ownsParam.shares)
RETURN mgr.managerName, owns.reportCalendarOrQuarter, com.companyName
"""

kg.query(cypher, params={ 'ownsParam': first_form13 })

[{'mgr.managerName': 'Royal Bank of Canada',
  'owns.reportCalendarOrQuarter': '2023-06-30',
  'com.companyName': 'NETAPP INC'}]

- 我们将运行一个快速查询再次检查，确保关系确实存在。
- 我们将在模式中抓取关系，然后从owns关系中返回股份和价值。

In [21]:
kg.query("""
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik})
-[owns:OWNS_STOCK_IN]->
        (com:Company {cusip6: $ownsParam.cusip6})
RETURN owns { .shares, .value }
""", params={ 'ownsParam': first_form13 })

[{'owns': {'shares': 842850, 'value': 64395000000.0}}]

- 完成一次后，我们可以遍历CSV文件中的所有行，创建管理公司和它们投资的公司之间的ownstock关系。
- 当然，那家公司将是NetApp。这和我们之前做的一样，我们只是遍历所有行，创建所有合并语句来创建关系。

In [22]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN { 
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter 
    }]->(com)
  ON CREATE
    SET owns.value  = toFloat($ownsParam.value), 
        owns.shares = toInteger($ownsParam.shares)
"""

# 循环所有的 Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'ownsParam': form13 })

- 再次快速检查确保我们做对了，我们预计创建了561个关系。

In [23]:
cypher = """
  MATCH (:Manager)-[owns:OWNS_STOCK_IN]->(:Company)
  RETURN count(owns) as investments
"""

kg.query(cypher)

[{'investments': 561}]

- 我们从最初开始就已经改变了知识图谱很多。我们从Form 10-Ks的块开始，然后我们连接这些块，并创建了一个表格以连接这些块。现在我们创建了公司和经理，所有这些都已连接。让我们看看知识图谱的模式，以了解我们所有工作的成果。
- 我们可以通过刷新知识图谱上的模式案来做到这一点，然后打印出模式。
- 我们将利用TextWrap来尝试获得一些良好的格式。

In [24]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties are the following: Chunk {textEmbedding:
LIST, f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
cik: STRING, cusip6: STRING, names: LIST, formId: STRING,
source: STRING, chunkId: STRING},Form {cusip6: STRING,
names: LIST, formId: STRING, source: STRING},Company
{cusip6: STRING, names: LIST, companyName: STRING, cusip:
STRING},Manager {managerName: STRING, managerCik: STRING,
managerAddress: STRING} Relationship properties are the
following: SECTION {f10kItem: STRING},OWNS_STOCK_IN {shares:
INTEGER, reportCalendarOrQuarter: STRING, value: FLOAT} The
relationships are the following: (:Chunk)-[:NEXT]-
>(:Chunk),(:Chunk)-[:PART_OF]->(:Form),(:Form)-[:SECTION]-
>(:Chunk),(:Company)-[:FILED]->(:Form),(:Manager)-
[:OWNS_STOCK_IN]->(:Company)


- 首先是我们在知识图谱中的节点。我们可以看到我们有一个带有属性的块节点，在这里是表格节点，它的属性。这里是我们创建的经理和公司及其所有属性。太棒了。
- 下半部分是关系，我们可以看到这里有一个块是表格的一部分。这是正确的。我们知道块之间有一个next关系。这是块的链表。太棒了。
- 我们也可以从表格通过部分到块。这就是我们找到链表开头的方式。
- 最后，我们知道经理拥有公司股票，并且公司提交了表格。
- 所有这些一起构成了我们刚刚创建的知识图谱。

# 四、图谱查询

让我们看看我们能在图谱中找到什么有趣的东西。

### 4.1 确定投资者数量

- 首先，找到一个随机块，以便在后续查询中使用。所以我们只是匹配块并返回第一个块ID作为chunk ID，并将其限制为1。
- 这是我们找到的第一个块。所以当我们运行这个查询时，我们会将其保存在chunk rows中并打印出chunk rows。

In [25]:
cypher = """
    MATCH (chunk:Chunk)
    RETURN chunk.chunkId as chunkId LIMIT 1
    """

chunk_rows = kg.query(cypher)
print(chunk_rows)

[{'chunkId': '0000950170-23-027948-item1-chunk0000'}]


- 很好，这就是我们的块ID。让我们存储它，以便以后使用。
- 我们可以看到这是一个包含字典的列表，所以首先让我们从列表中提取内容。所以现在我们只有字典。

In [26]:
chunk_first_row = chunk_rows[0]
print(chunk_first_row)

{'chunkId': '0000950170-23-027948-item1-chunk0000'}


- 然后，我们从块的第一行中提取块ID并将其存储在refChunkId中，以便以后查询使用。

In [27]:
ref_chunk_id = chunk_first_row['chunkId']
print(ref_chunk_id)

'0000950170-23-027948-item1-chunk0000'

- 我们将从我们知道的块开始，逐步通过表格回到其他可以逐步发现的东西，看看可能是什么。
- 首先，我们将从我们知道的块开始，块ID，然后通过part of关系到它所属的表格，并返回表格来源。

In [28]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form)
    RETURN f.source
    """

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}]

- 我们将再进行一步，在这个模式中，我们将实际上把它分成两行。所以这是一个块，它是一个表格的一部分，而那个表格是由公司提交的，如果我们倒过来看第二部分，并返回公司名称。
- 正如我们所期望的，这是我们的好朋友 NetApp 。

In [29]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
    (com:Company)-[:FILED]->(f)
RETURN com.companyName as name
"""

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'name': 'NETAPP INC'}]

- 我们将再扩展一步。这里我们将从一个块开始，它是表格的一部分，而那个公司提交了表格。然后有一个经理或管理投资公司拥有那个公司的股票。这里的 com 就是公司，所有这些变量必须匹配。
- 这是一个大的模式，分成三部分。我们将返回公司名称和投资该公司的经理数量，称其为投资者数量。
- 这是一个很好的验证。好的，当然公司是NetApp，正如预期，我们知道有561个投资者。

In [30]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[:OWNS_STOCK_IN]->(com)
RETURN com.companyName, 
        count(mgr.managerName) as numberOfinvestors 
LIMIT 1
"""

kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'com.companyName': 'NETAPP INC', 'numberOfinvestors': 561}]

### 4.2 使用查询为 LLM 构建额外的上下文

- 我们开始看到一些知识图谱的有趣且有用之处，这是仅通过向量搜索无法实现的信息。
- 我们刚刚创建的模式是有用的信息，我们可以使用这些信息来扩展提供给LLM的上下文。例如，我们可以找到公司的投资者，然后创建包含每个投资的详细信息的句子，以进一步扩展提供给LLM的信息。
- 我们将使用之前的相同匹配，相同的模式，然后不仅仅是返回那些数据，而是将其中一些数据转换为字符串句子。
- 我们会注意到两件额外的事情。这只是将字符串连接在一起，但这里我们将值转化为整数并使用 APOC 数字格式进行格式化，这将添加一些逗号以提高可读性。
- 让我们将其保存到结果中，看看我们是否可以提取句子，让我们读得更清楚。

In [31]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
    RETURN mgr.managerName + " owns " + owns.shares + 
        " shares of " + com.companyName + 
        " at a value of $" + 
        apoc.number.format(toInteger(owns.value)) AS text
    LIMIT 10
    """
kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'text': 'CSS LLC/IL owns 12500 shares of NETAPP INC at a value of $955,000,000'},
 {'text': 'BOKF, NA owns 40774 shares of NETAPP INC at a value of $3,115,134,000'},
 {'text': 'BANK OF NOVA SCOTIA owns 18676 shares of NETAPP INC at a value of $1,426,847,000'},
 {'text': 'Jefferies Financial Group Inc. owns 23200 shares of NETAPP INC at a value of $1,772,480,000'},
 {'text': 'DEUTSCHE BANK AG\\ owns 929854 shares of NETAPP INC at a value of $71,040,845,000'},
 {'text': 'TORONTO DOMINION BANK owns 183163 shares of NETAPP INC at a value of $13,984,000'},
 {'text': 'STATE BOARD OF ADMINISTRATION OF FLORIDA RETIREMENT SYSTEM owns 265756 shares of NETAPP INC at a value of $20,303,759,000'},
 {'text': 'NISA INVESTMENT ADVISORS, LLC owns 67848 shares of NETAPP INC at a value of $5,183,587,000'},
 {'text': 'ONTARIO TEACHERS PENSION PLAN BOARD owns 7290 shares of NETAPP INC at a value of $556,956,000'},
 {'text': 'STATE STREET CORP owns 9321206 shares of NETAPP INC at a value of $712,140,138,0

- 看看我们创建的第一个句子，我们可以看到这个基金名为company，拥有大量NetApp股票，价值非常大。
- 我们可以看到使用模式匹配并将这些模式的值转换为句子所能做的事情，让我们在 RAG 工作流中运用这一点。

In [32]:
results = kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})
print(textwrap.fill(results[0]['text'], 60))

CSS LLC/IL owns 12500 shares of NETAPP INC at a value of
$955,000,000


## 4.3 RAG 工作流

- 我们像之前一样设置两个不同的 LangChain 链。一个只是常规的向量检索，另一个将有一个检索查询以获取额外信息。我们将对一些问题进行测试。
- 第一个链称为plain chain，这将是仅进行向量搜索的链。

In [33]:
vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)
# 从向量存储中创建一个检索器
retriever = vector_store.as_retriever()

# 从检索器中创建一个聊天机器人问答链
plain_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever
)

- 然后我们将定义一个 Cypher 查询，我们希望扩展向量搜索。所以我们将其称为投资检索查询。
- 这种模式现在应该看起来很熟悉。从一个特定的节点开始，记住这将来自向量搜索。它将给我们一个找到的节点，该节点类似于提出的问题。我们将从那个节点开始，知道它是一个表格的一部分。
- 从表格中，我们知道它是由某家公司提交的。有一个经理拥有那家公司的股票。箭头指向另一个方向，所以我们必须倒过来看这些。
- 从原始节点，我们还将获取分数、经理、owns关系和公司。我们将根据投资股份降序排列所有这些，并仅限于10个。
- 然后我们将基本上创建一个包含所有内容的巨大的文本，收集所有内容并将其放入一些语句中，然后将其添加到我们将从向量搜索中返回的文本中。

In [34]:
investment_retrieval_query = """
MATCH (node)-[:PART_OF]->(f:Form),
    (f)<-[:FILED]-(com:Company),
    (com)<-[owns:OWNS_STOCK_IN]-(mgr:Manager)
WITH node, score, mgr, owns, com 
    ORDER BY owns.shares DESC LIMIT 10
WITH collect (
    mgr.managerName + 
    " owns " + owns.shares + 
    " shares in " + com.companyName + 
    " at a value of $" + 
    apoc.number.format(toInteger(owns.value)) + "." 
) AS investment_statements, node, score
RETURN apoc.text.join(investment_statements, "\n") + 
    "\n" + node.text AS text,
    score,
    { 
      source: node.source
    } as metadata
"""

- 像前面的课程一样，我们将创建一个新的向量存储，但现在使用扩展查询。
- 这里的检索查询是我们刚刚定义的投资检索查询。
- 我们将从中创建一个检索器，然后是一个链。我们称之为投资链。

In [35]:
vector_store_with_investment = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=investment_retrieval_query,
)

# 从向量存储中创建一个检索器
retriever_with_investments = vector_store_with_investment.as_retriever()

# 从检索器中创建一个聊天机器人问答链
investment_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever_with_investments
)

- 现在，我们准备尝试几个不同的问题。让我们从一个显而易见的问题开始，因为我们知道这都是关于NetApp的。
- 我们会问一个问题：用一句话告诉我关于NetApp的情况。我们将使用我们刚刚定义的plain chain，它只是进行向量搜索。我们先运行这个。
- 我们可以看到这里，NetApp是一家全球领先的云公司。

In [36]:
question = "In a single sentence, tell me about Netapp."

In [ ]:
question_zh = "用一句话告诉我有关 Netapp 的信息。"

In [37]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp is a global cloud-led, data-centric software company that provides customers the freedom to manage applications and data across hybrid multicloud environments. \n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

- 让我们用投资链尝试同样的事情。看看那个额外的上下文是否会改变对NetApp的总结。
- 这实际上看起来很相似，这并不意外。LLM忽略了关于投资的额外信息，因为我们并没有真的问到这点。

In [38]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp is a global cloud-led, data-centric software company that focuses on enterprise storage and data management, cloud storage, and cloud operations markets, providing customers with the freedom to manage applications and data across hybrid multicloud environments. \n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

如果我们确实问到投资者呢？

In [39]:
question = "In a single sentence, tell me about Netapp investors."

In [ ]:
question_zh = "用一句话告诉我有关 Netapp 投资者的信息。"

- 我们将先从plain chain开始。这只会进行向量搜索，看看我们得到什么答案。
- 它认为投资者是一个多元化的客户基础。它只是试图将事情拼凑在一起，试图给出某种答案。

In [40]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'Netapp investors are diverse and include global enterprises, local businesses, and government installations who look to NetApp and its ecosystem of partners to maximize the business value of their IT and cloud investments.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

- 所以现在我们用投资链问同样的问题，看看我们得到什么答案。
- 这更现实一点。现在我们可以看到投资者包括 Vanguard 集团，一些财务主管，剑桥投资。

In [41]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'Netapp investors include Vanguard Group Inc., BlackRock Inc., and PRIMECAP MANAGEMENT CO/CA/.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

这实际上是一个很好的开始。我们可以更改几件不同的事情。
- 我们可以更改我们从投资中创建的句子，看看这如何影响我们得到的内容。
- 改变我们在最后问的问题，看看不同的提示如何调整我们从LLM中得到的内容。

在让LLM理解我们提供的信息以及这些信息与可以回答的问题之间的关系上，还有一些艺术性。